# NBA Lineup Selector
Author: Kahlil Gedin <br>
Date: 12/24/19

## import modules

In [1]:
import numpy as np
import pandas as pd
import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


import time

timestr =time.strftime("%d%m%y")

In [2]:
def clean1(self):
    self.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
    self.drop(["a"], axis=1, inplace=True)
    
def clean2(self):
    self.rename({"Unnamed: 0.1":"a"}, axis="columns", inplace=True)
    self.drop(["a"], axis=1, inplace=True)

In [3]:
def pdfA(x, mean, std):
    b=[]
    for i in range(len(x)):
        b.append((1/(std*math.sqrt(2*math.pi)))*math.exp((-(x[i]-mean)**2)/(2*std**2)))
    return b
def pdfO(x, mean, std):
    b=[]
    for i in range(len(x)):
        b.append((1/(std*math.sqrt(2*math.pi)))*math.exp((-(x[i]-mean)**2)/(2*std**2))*2.5)
    return b

def pdfT(x, mean, std):
    b=[]
    for i in range(len(x)):
        b.append((1/(std*math.sqrt(2*math.pi)))*math.exp((-(x[i]-mean)**2)/(2*std**2))*18000)
    return b

def pdfV(x, mean, std):
    b=[]
    for i in range(len(x)):
        b.append((1/(std*math.sqrt(2*math.pi)))*math.exp((-(x[i]-mean)**2)/(2*std**2))*20000)
    return b

In [4]:
def norm(x,y): 
  return (x - y['mean']) / y['std']

## load model

In [5]:
model =  keras.models.load_model('luEval3.h5')

## format data

In [6]:
data = pd.read_csv('gpp{date}.csv'.format(date=timestr))
datal =data.drop('Line#', axis=1)

In [7]:
data_stats = datal.describe()
data_stats = data_stats.transpose()
ndata_data = norm(datal, data_stats)

In [8]:
datal

,Top1,Top1%,Top10,Top10%,Bottom,Bottom%,ROI,ceil,AvgPt,std,odds
0,5,0.5,101,10.1,899,89.9,-25880,381.6862,282.572101,37.616884,3.150236
1,6,0.6,105,10.5,895,89.5,-25000,374.4681,284.296866,37.128074,0.858207
2,3,0.3,59,5.9,941,94.1,-31720,376.9326,276.879421,36.987336,0.208406
3,6,0.6,89,8.9,911,91.1,-26920,381.7414,281.574268,38.744359,0.105033
4,8,0.8,78,7.8,922,92.2,-27440,379.0209,276.946379,37.718521,0.848304
...,...,...,...,...,...,...,...,...,...,...,...
995,13,1.3,137,13.7,863,86.3,-18360,377.6095,283.694458,39.131990,2.577610
996,6,0.6,62,6.2,938,93.8,-30160,363.6651,270.010783,37.982931,0.792098
997,11,1.1,81,8.1,919,91.9,-25880,372.1620,276.744518,40.349897,0.274758
998,17,1.7,124,12.4,876,87.6,-18320,383.0121,283.739712,39.510255,1.206818


## make predictions

In [9]:
predictions = model.predict(ndata_data)

In [10]:
# if there is a column with final use this
#data.insert(13, 'predict', predictions)

data.insert(11, 'predict', predictions)

In [11]:
#data.insert(12, 'stat', pdfV(datal.predict, datal.predict.mean(), datal.predict.std()))

In [12]:
data

,Line#,Top1,Top1%,Top10,Top10%,Bottom,Bottom%,ROI,ceil,AvgPt,std,predict,odds
0,1,5,0.5,101,10.1,899,89.9,-25880,381.6862,282.572101,37.616884,130.317459,3.150236
1,2,6,0.6,105,10.5,895,89.5,-25000,374.4681,284.296866,37.128074,408.572693,0.858207
2,3,3,0.3,59,5.9,941,94.1,-31720,376.9326,276.879421,36.987336,329.324402,0.208406
3,4,6,0.6,89,8.9,911,91.1,-26920,381.7414,281.574268,38.744359,183.674774,0.105033
4,5,8,0.8,78,7.8,922,92.2,-27440,379.0209,276.946379,37.718521,243.077072,0.848304
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,13,1.3,137,13.7,863,86.3,-18360,377.6095,283.694458,39.131990,175.888428,2.577610
996,997,6,0.6,62,6.2,938,93.8,-30160,363.6651,270.010783,37.982931,309.485413,0.792098
997,998,11,1.1,81,8.1,919,91.9,-25880,372.1620,276.744518,40.349897,223.970306,0.274758
998,999,17,1.7,124,12.4,876,87.6,-18320,383.0121,283.739712,39.510255,270.453003,1.206818


## filter results

In [13]:
#changed lower value from 135 to 200

lo= data[(data.predict >= 200)]
lo= lo[lo.predict <= 327]
lo = lo[lo.odds>= 0.0038]
#lo = lo[lo.AvgPt>= 369]
lo = lo[lo.AvgPt<= 410]
#lo = lo[lo.stat>= 200]

In [14]:
optimal = lo.sort_values(by='predict', ascending = False)[:301]

In [15]:
optimal.sort_values(by='Line#', ascending = True)

,Line#,Top1,Top1%,Top10,Top10%,Bottom,Bottom%,ROI,ceil,AvgPt,std,predict,odds
16,17,17,1.7,128,12.8,872,87.2,-17840,382.0965,283.017281,42.854954,277.065582,0.147625
20,21,13,1.3,145,14.5,855,85.5,-17400,384.3144,286.838545,40.784979,326.151855,1.640031
21,22,9,0.9,74,7.4,926,92.6,-27520,370.9505,280.506413,39.102435,306.944916,0.138871
24,25,5,0.5,86,8.6,914,91.4,-27680,370.3347,279.776280,36.544016,297.843994,0.200138
25,26,16,1.6,98,9.8,902,90.2,-21840,378.6060,280.912755,38.072208,277.101471,0.195123
...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,983,3,0.3,91,9.1,909,90.9,-27880,386.4963,281.276643,38.594929,303.715912,0.209791
988,989,12,1.2,95,9.5,905,90.5,-23800,378.3075,280.059222,39.182513,289.405090,1.230044
993,994,9,0.9,100,10.0,900,90.0,-24400,376.2888,280.586830,40.311822,305.720612,0.143131
996,997,6,0.6,62,6.2,938,93.8,-30160,363.6651,270.010783,37.982931,309.485413,0.792098


# save results

In [16]:
optimal.to_csv('f_line{date}.csv'.format(date=timestr), index=0)
#optimal.to_csv('f_line240120.csv', index=0)


In [17]:
optimal

,Line#,Top1,Top1%,Top10,Top10%,Bottom,Bottom%,ROI,ceil,AvgPt,std,predict,odds
236,237,14,1.4,122,12.2,878,87.8,-19760,388.8324,286.136307,42.176472,326.152252,0.620540
20,21,13,1.3,145,14.5,855,85.5,-17400,384.3144,286.838545,40.784979,326.151855,1.640031
131,132,6,0.6,96,9.6,904,90.4,-26080,369.3722,279.125041,37.431049,326.078888,0.558822
167,168,4,0.4,82,8.2,918,91.8,-28560,372.3277,280.200062,36.696060,326.007416,0.290589
273,274,13,1.3,144,14.4,856,85.6,-17520,393.0155,288.610672,40.554660,325.723053,0.153661
...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,881,18,1.8,137,13.7,863,86.3,-16360,394.7688,286.119432,41.265844,270.579803,0.332460
539,540,9,0.9,104,10.4,896,89.6,-23920,374.5448,281.492470,40.135890,270.517944,0.539271
998,999,17,1.7,124,12.4,876,87.6,-18320,383.0121,283.739712,39.510255,270.453003,1.206818
284,285,16,1.6,115,11.5,885,88.5,-19800,385.4025,281.723721,39.833760,270.365875,0.936416
